<a href="https://colab.research.google.com/github/pwaila/torchlearn/blob/master/assignment2/FirstNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Torch and Torchvision libraries

In [2]:
!pip install torch
!pip install torchvision


    100% |████████████████████████████████| 591.8MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61c52000 @  0x7fbb7b58c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 3.8MB/s 
    100% |████████████████████████████████| 2.0MB 13.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:

import torch as T
import torchvision as TV
from torchvision import datasets, transforms

In [4]:
dataset_path = './cifar10'

transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_loader = T.utils.data.DataLoader(datasets.CIFAR10(root=dataset_path, train=True, download=True,transform=transform_train))
test_loader = T.utils.data.DataLoader(datasets.CIFAR10(root=dataset_path, train=False, download=True,transform=transform_test))


Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Convolutional neural network

In [0]:

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()


Defining a loss function

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


Training

In [8]:

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.305
[1,  4000] loss: 2.260
[1,  6000] loss: 2.119
[1,  8000] loss: 2.012
[1, 10000] loss: 1.967
[1, 12000] loss: 1.895
[1, 14000] loss: 1.861
[1, 16000] loss: 1.818
[1, 18000] loss: 1.738
[1, 20000] loss: 1.754
[1, 22000] loss: 1.718
[1, 24000] loss: 1.664
[1, 26000] loss: 1.662
[1, 28000] loss: 1.674
[1, 30000] loss: 1.676
[1, 32000] loss: 1.640
[1, 34000] loss: 1.725
[1, 36000] loss: 1.678
[1, 38000] loss: 1.649
[1, 40000] loss: 1.650
[1, 42000] loss: 1.613
[1, 44000] loss: 1.672
[1, 46000] loss: 1.618
[1, 48000] loss: 1.617
[1, 50000] loss: 1.614
[2,  2000] loss: 1.589
[2,  4000] loss: 1.615
[2,  6000] loss: 1.622
[2,  8000] loss: 1.606
[2, 10000] loss: 1.589
[2, 12000] loss: 1.588
[2, 14000] loss: 1.600
[2, 16000] loss: 1.607
[2, 18000] loss: 1.549
[2, 20000] loss: 1.558
[2, 22000] loss: 1.594
[2, 24000] loss: 1.542
[2, 26000] loss: 1.562
[2, 28000] loss: 1.551
[2, 30000] loss: 1.576
[2, 32000] loss: 1.527
[2, 34000] loss: 1.626
[2, 36000] loss: 1.569
[2, 38000] 

Test

In [11]:
correct = 0
total = 0
with T.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = T.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 44 %


In [0]:

import torch.nn as nn
import torch.nn.functional as F


class Net2(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
